# Assistant de recherche et de synthèse d'articles scientifiques

L’objectif de ce projet est de concevoir un assistant capable d’effectuer des recherches sur des articles scientifiques, en identifiant les publications les plus pertinentes, puis en générant une synthèse de chacun d’eux afin de fournir une réponse claire et structurée.


In [11]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_tavily import TavilySearch
from langchain_community.retrievers import ArxivRetriever
from langchain_community.retrievers import PubMedRetriever
from typing import List, TypedDict, Annotated
import operator
import os
from dotenv import load_dotenv


In [30]:
load_dotenv()
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
KEY_API= os.getenv("GROQ_API_KEY")

In [31]:
llm = ChatGroq(model = "llama-3.3-70b-versatile", temperature=0)

In [ ]:
class AgentState(TypedDict):

    messages: Annotated[List[AnyMessage], operator.add]

@tool
def tavily(message):
    """Moteur de recherche en ligne en temps réel
    
    Args:
       message: la requête de l'utilisateur"""
    
    tavily = TavilySearch(max_results = 5, topic = "science")

    return tavily.invoke({"query": message})

@tool
def arxiv(message):
    """Acceder à des articles dans le domaine de la science
    
    Args:
       message: la requête de l'utilisateur
    """

    arxiv = ArxivRetriever(
    load_max_docs=5,
    get_ful_documents=True)

    return arxiv.invoke(message)

@tool 
def pubmed(message):
    """Accéder à des citations dans le domaine de la medecine
    
    Args: 
       message: la requête de l'utilisateur"""
    
    pubmed = PubMedRetriever()

    return pubmed.invoke(message)
    
tools = [tavily, arxiv, pubmed]

llm_with_tools = llm.bind_tools(tools)